<a href="https://colab.research.google.com/github/simransirsat/MLPracticals/blob/main/Information_Gain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**5A Information Gain**

In [6]:
pip install anytree

In [7]:
import pandas as pd
from math import log2

In [8]:
from anytree import Node, RenderTree

In [9]:
df = pd.read_csv('/content/sample_data/data.csv')

In [10]:
def Info(df):
    possible_labels = {}
    for i in range(0, df.shape[0]):
        if df.iloc[i, -1] not in possible_labels:
            possible_labels[df.iloc[i, -1]] = 1
        else:
            possible_labels[df.iloc[i, -1]] += 1
    info = 0
    total_records = df.shape[0]
    for label, count in possible_labels.items():
        info -= (count / total_records) * log2((count / total_records))

    assert (info >= 0), "Info(D) is negative"
    return [ round(info,4), possible_labels]

def entropy(df, attr, attr_label, possible_values):
    columns = {}
    for i in range(0, len(list(df.columns))):
        columns[list(df.columns)[i]] = i

    label_cnts = {}
    for i in range(0, df.shape[0]):
        if(df.iloc[i, columns[attr]] == attr_label):
            if df.iloc[i, -1] not in label_cnts:
                label_cnts[df.iloc[i, -1]] = 1
            else:
                label_cnts[df.iloc[i, -1]] += 1
    info = 0
    for label, count in label_cnts.items():
        info -= (count / possible_values[attr_label]) * log2((count / possible_values[attr_label]))
    return round(info,4)

def Info_attr(df, attr):
    columns = {}
    for i in range(0, len(list(df.columns))):
        columns[list(df.columns)[i]] = i

    possible_values = {}        # (attr_labels, cnt) : (<30. 4)
    for i in range(0, df.shape[0]):
        if df.iloc[i, columns[attr]] not in possible_values:
            possible_values[df.iloc[i, columns[attr]]] = 1
        else:
            possible_values[df.iloc[i, columns[attr]]] += 1
    
    info_attr = 0
    for attr_label, cnt in possible_values.items():
        info_attr += (cnt / df.shape[0]) * entropy(df, attr, attr_label, possible_values)
        print("\t", attr_label, "->", entropy(df, attr, attr_label, possible_values))
    return round(info_attr,4)

def find_deciding_attr(df):
    max_gain = float('-inf')
    max_gain_attr = None

    print('Info_df:\t', Info(df)[0])
    for i in range(0, df.shape[1] - 1):
        print('\n', df.columns[i])
        info_attr = Info_attr(df, df.columns[i])
        print('Info(', df.columns[i],') :', info_attr)
        gain = round(Info(df)[0] - info_attr, 4)
        print('GAIN:\t', gain)
        if(gain > max_gain):
            max_gain = gain
            max_gain_attr = df.columns[i]
    if max_gain_attr is None:
        print(df)
    assert max_gain_attr is not None, "Attribute finding error, max_gain_attr not set"
    print('\n\nDeciding Attribute: ', max_gain_attr,' with gain = ', round(max_gain,4))

    return max_gain_attr

root = None

def make_decision_tree(df, parent = None, weight = None):
    global root    
    possible_target = {}
    for i in range(0, df.shape[0]):
        if df.iloc[i, -1] not in possible_target:
            possible_target[df.iloc[i, -1]] = True
    
    if (len(possible_target) == 1):
        node = Node(list(possible_target)[0], parent = parent, weight = str(weight))
        return

    if(df.shape[1] == 1 and len(possible_target) > 1):
        node = Node(list(possible_target)[0], parent = parent, weight = str(weight))
        return
    
    deciding_attr = find_deciding_attr(df)
    node = Node(deciding_attr, parent = parent, weight = str(weight))
    #print('df:', df)
    if (root is None):
        root = node
    
    columns = {}
    for i in range(0, len(list(df.columns))):
        columns[list(df.columns)[i]] = i
    
    possible_values = {}
    for i in range(0, df.shape[0]):
        if df.iloc[i, columns[deciding_attr]] not in possible_values:
            possible_values[df.iloc[i, columns[deciding_attr]]] = True

    #print('possible_values: ', possible_values)
    for attr_wt_label, _ in possible_values.items():
        print('Now checking for ',deciding_attr,':\t', attr_wt_label)
        cols = list(df.columns)
        cols.remove(deciding_attr)
        df_temp = pd.DataFrame(columns = cols)
        
        for i in range(0, df.shape[0]):
            if (df.iloc[i, columns[deciding_attr]] == attr_wt_label):
                row = df.iloc[i, :].tolist()
                row.pop(columns[deciding_attr])
                #df_temp.loc[len(df)] = row
                df_temp = df_temp.append(pd.DataFrame([row], columns = cols), ignore_index = True)
        #print('Shape:\t'. df_temp.shape)

        make_decision_tree(df_temp, node, attr_wt_label)

    return

In [11]:
root = None
make_decision_tree(df)

print('The Decision Tree is:')
for pre, _, node in RenderTree(root):
    print("%s%s (%s)" % (pre, node.name, node.weight))

Info_df:	 0.9799

 Restaurant
	 Sam's -> 0.971
	 Kim's -> 0.9852
Info( Restaurant ) : 0.9793
GAIN:	 0.0006

 Meal
	 Breakfast -> 0.9852
	 Lunch -> 0.971
Info( Meal ) : 0.9793
GAIN:	 0.0006

 Day
	 Saturday -> 0.9183
	 Sunday -> 0.9183
	 Friday -> 0.9183
Info( Day ) : 0.9183
GAIN:	 0.0616

 Cost
	 Expensive -> 0.7219
	 Cheap -> 0.5917
Info( Cost ) : 0.646
GAIN:	 0.3339

 Vegan
	 Yes -> 1.0
	 No -> 0.9544
Info( Vegan ) : 0.9696
GAIN:	 0.0103


Deciding Attribute:  Cost  with gain =  0.3339
Now checking for  Cost :	 Expensive
Info_df:	 0.7219

 Restaurant
	 Sam's -> 0.0
	 Kim's -> 0.8113
Info( Restaurant ) : 0.649
GAIN:	 0.0729

 Meal
	 Breakfast -> 0.8113
	 Lunch -> 0.0
Info( Meal ) : 0.649
GAIN:	 0.0729

 Day
	 Saturday -> 0.0
	 Sunday -> 1.0
	 Friday -> 0.0
Info( Day ) : 0.4
GAIN:	 0.3219

 Vegan
	 Yes -> 0.0
	 No -> 0.8113
Info( Vegan ) : 0.649
GAIN:	 0.0729


Deciding Attribute:  Day  with gain =  0.3219
Now checking for  Day :	 Saturday
Now checking for  Day :	 Sunday
Info_df:	 1.0
